In [20]:
import numpy as np
import pandas as pd

In [21]:
spread_prob_df = pd.concat([pd.read_csv('../spread_probability_top30/%d.csv'%i) for i in range(1,5)],ignore_index=True)
spread_prob_df['from'] = spread_prob_df['from'].apply(lambda x: '%012d'%x)
spread_prob_df['to'] = spread_prob_df['to'].apply(lambda x: '%012d'%x)
spread_prob_df = spread_prob_df[['from', 'to', 'prob']].reset_index(drop = True)

In [22]:
import random

In [23]:
random.seed(42)
np.random.seed(42)

In [24]:
pop_data = pd.read_csv('../src_data/usa_population_revise.csv', dtype ={'GeoId':str, 'Population':np.int64})

In [25]:
pop_data

,GeoId,Population
0,010010201001,730
1,010010201002,1263
2,010010202001,835
3,010010202002,1124
4,010010203001,2774
...,...,...
220329,721537506011,883
220330,721537506012,2523
220331,721537506013,991
220332,721537506021,1577


In [26]:
starts = list(spread_prob_df['from'].unique())
random.shuffle(starts)
starts = starts[:5]

In [27]:
def init_counter():
    counter = {}
    total_infected_tracker = {}
    N = len(pop_data)
    for i in range(N):
        cbg = pop_data.loc[i,'GeoId']
        pop = int(pop_data.loc[i,'Population'])
        counter[cbg] = [pop, 0, 0] # S, I, R
        total_infected_tracker[cbg] = [pop, 0] # population, total infected
    return counter, total_infected_tracker

In [30]:
num_init_cases = 500
infectious_rate = [0.1,0.2,0.3,0.3,0.2,0.1,0.1]
infectious_day = len(infectious_rate)
spread_prob_grouped_df = spread_prob_df.groupby('from')
active_case = {}
counter,total_case_tracker = init_counter()
region_level_output_str = 'Day,GeoId,Susceptible,Infectious,Recovered\n'
case_level_output_str = 'Day,from_GeoId,from_case,to_GeoId,to_case\n'
simulation_day = 0
# random initiate spread in 5 regions
for s in starts:
    num_init_cases_refine = min(counter[s][0], num_init_cases)
    active_case[s] = [[0] * infectious_day, [0] * infectious_day] # [case/day, idx/day]
    active_case[s][0][0] = num_init_cases_refine
    counter[s][0] -= num_init_cases_refine
    counter[s][1] += num_init_cases_refine
    total_case_tracker[s][1] += num_init_cases_refine
    # region level output update
    # header: Day, GeoId, Susceptible, Infectious, Recovered
    region_level_output_str += '%d,%s,%d,%d,%d\n'%(simulation_day, s, counter[s][0], counter[s][1], counter[s][2])

    # case level output update
    # header: Day, from_GeoId, from_case, to_GeoId, to_case
    # simulation initialed: 000..0, -1
    simu_GeoId = '0' * len(s)
    for i in range(num_init_cases_refine):
        case_level_output_str += '%d,%s,%d,%s,%d\n'%(simulation_day,simu_GeoId, -1, s, i)

# save the outputs
# Todo

In [31]:
print(case_level_output_str)

Day,from_GeoId,from_case,to_GeoId,to_case
0,000000000000,-1,250092524002,0
0,000000000000,-1,250092524002,1
0,000000000000,-1,250092524002,2
0,000000000000,-1,250092524002,3
0,000000000000,-1,250092524002,4
0,000000000000,-1,250092524002,5
0,000000000000,-1,250092524002,6
0,000000000000,-1,250092524002,7
0,000000000000,-1,250092524002,8
0,000000000000,-1,250092524002,9
0,000000000000,-1,250092524002,10
0,000000000000,-1,250092524002,11
0,000000000000,-1,250092524002,12
0,000000000000,-1,250092524002,13
0,000000000000,-1,250092524002,14
0,000000000000,-1,250092524002,15
0,000000000000,-1,250092524002,16
0,000000000000,-1,250092524002,17
0,000000000000,-1,250092524002,18
0,000000000000,-1,250092524002,19
0,000000000000,-1,250092524002,20
0,000000000000,-1,250092524002,21
0,000000000000,-1,250092524002,22
0,000000000000,-1,250092524002,23
0,000000000000,-1,250092524002,24
0,000000000000,-1,250092524002,25
0,000000000000,-1,250092524002,26
0,000000000000,-1,250092524002,27
0,000000000000,-

In [32]:
def get_random_des(des_prob_df):
    rand_val = random.random()
    cumprob = des_prob_df['prob'].cumsum()
    for des, prob in zip(des_prob_df['to'], cumprob):
        if rand_val < prob: return des
    return des_prob_df.loc[-1,'to']

In [173]:
def next_day_new(simulation_day):
    region_level_output_str = ''
    case_level_output_str = ''
    region_recover_str = ''
    new_cases = {}
    for cbg in list(active_case.keys()):
        
        des_prob_df = spread_prob_grouped_df.get_group(cbg)
        cbg_active_case = active_case[cbg][0]
        cbg_case_idx_lower = active_case[cbg][1]
        src_cbg_idx = counter[counter['GeoId'] == cbg].index
        print(cbg_active_case)
        for d in range(infectious_day-1, -1, -1):

            num_infectious = cbg_active_case[d]
            current_case_idx = cbg_case_idx_lower[d]

            # update for recovered cases
            if d == infectious_day-1:
                counter[cbg][2] += num_infectious
                counter[cbg][1] -= num_infectious
                # update region level output on recovered cases
                region_recover_str += '%d,%s,%d,%d,%d\n'%(simulation_day+1, cbg, counter[cbg][0], counter[cbg][1], counter[cbg][2])
                
            if d != 0:
                cbg_active_case[d] = cbg_active_case[d-1]
                cbg_case_idx_lower[d] = cbg_case_idx_lower[d-1]
            else:
                cbg_active_case[d] = 0

            if num_infectious == 0: continue
                
            # calculate number of expected new cases in infectious day 'd'
            expected_new_case = num_infectious * infectious_rate[d]

            new_case_by_day = {}
            actual_new_case = 0
            if expected_new_case < 30:
                if num_infectious < 30: # naive approach
                    for idx in range(num_infectious):
                        if random.random() < infectious_rate[d]:
                            des_cbg = get_random_des(des_prob_df)
                            reduce_prob = total_case_tracker[des_cbg][1]/total_case_tracker[des_cbg][0]
                            if random.random() < reduce_prob: continue # Assume spread to an already infected person
                            actual_new_case += 1
                            if des_cbg not in new_case_by_day:
                                new_case_by_day[des_cbg] = 0
                            new_case_by_day[des_cbg] += 1
                            total_case_tracker[des_cbg][1] += 1
                            
                            # update case level output
                            case_level_output_str += '%d,%s,%d,%s,%d\n'%(simulation_day, cbg, idx+current_case_idx, des_cbg, total_case_tracker[des_cbg]-1)

                else: # normal distribution on number of new cases, then naive approach for destinations
                    actual_new_case = np.random.normal(expected_new_case, expected_new_case * (1-infectious_rate[d]))
                    actual_new_case = int(max(0,actual_new_case))
                    failed_spread = 0
                    for _ in range(actual_new_case):
                        des_cbg = get_random_des(des_prob_df)
                        reduce_prob = total_case_tracker[des_cbg][1]/total_case_tracker[des_cbg][0]
                        if random.random() < reduce_prob: # Assume spread to an already infected person
                            failed_spread += 1
                            continue
                        if des_cbg not in new_case_by_day:
                            new_case_by_day[des_cbg] = 0
                        new_case_by_day[des_cbg] += 1
                        total_case_tracker[des_cbg][1] += 1
                    actual_new_case -= failed_spread

            else: # calculate expected number of new cases, then use normal/Poisson distribution for destinations
                for des_idx in des_prob_df.index:
                    des_cbg = des_prob_df[des_idx]['to']
                    des_susceptible = counter[des_cbg][0]
                    if des_susceptible == 0: continue

                    des_prob = des_prob_df[des_idx]['prob']
                    expected_new_case_in_des = expected_new_case * des_prob

                    if expected_new_case_in_des < 5:
                        actual_new_case_in_des = np.random.poisson(expected_new_case_in_des)
                    else:
                        actual_new_case_in_des = np.random.normal(expected_new_case_in_des, expected_new_case_in_des* (1-des_prob))
                        actual_new_case_in_des = int(max(0,actual_new_case_in_des))


                    actual_new_case += actual_new_case_in_des
                    if des_cbg not in new_case_by_day:
                        new_case_by_day[des_cbg] = 0
                    new_case_by_day[des_cbg] += actual_new_case_in_des

            num_new_cases = 0
            if num_infectious < 30:
                for case_idx in range(num_infectious): 
                    if random.random() < infectious_rate[d]: num_new_cases += 1
            else:
                expected_cases = infectious_rate[d]*num_infectious
                if expected_cases >= 5:
                    num_new_cases = int(np.floor(max(np.random.normal(expected_cases,expected_cases*(1-infectious_rate[d])), 0)))
                else:
                    num_new_cases = np.random.poisson(expected_cases)
            if num_new_cases == 0: continue

            # calculate number of new cases in each possible infected cbg
            new_cases_by_infectious_day = {}
            num_new_cases_actual = 0
            if num_new_cases < 30: # Number of cases too small to use statistical methods
                for i in range(num_new_cases):
                    des_cbg = get_random_des(des_prob_df)
                    # Spread to a fully infected region, assume nothing happens
                    if counter[counter['GeoId'] == des_cbg]['susceptible'].item() == 0: continue
                    if des_cbg not in new_cases_by_infectious_day: new_cases_by_infectious_day[des_cbg] = 0
                    new_cases_by_infectious_day[des_cbg] += 1
                    num_new_cases_actual += 1
            else: # Approximate Binomial distribution with Normal distribution and Poisson distribution
                for i in des_prob_df.index:
                    des = des_prob_df.loc[i, 'to']
                    des_s_num = int(counter[counter['GeoId'] == des]['susceptible'].item())
                    if des_s_num == 0: continue
                    
                    prob = des_prob_df.loc[i, 'prob']
                    expected_cases = num_new_cases * prob
                    expected_no_cases =  num_new_cases * (1-prob)
                    if expected_cases > 5 and expected_no_cases > 5:
                        var_case = expected_cases * (1-prob)
                        des_new_case = int(np.floor(np.random.normal(expected_cases,var_case)))
                    else:
                        des_new_case = np.random.poisson(expected_cases)
                        
                    # Check if fully infected
                    des_new_case = min(des_s_num, des_new_case)
                    des_new_case = max(0,des_new_case)
                    if des_new_case > 0: new_cases_by_infectious_day[des] = des_new_case
                    num_new_cases_actual += des_new_case
            src_case_id = cbg_case_idx_lower[d] + np.random.choice(num_infectious, size = num_new_cases_actual, replace = False)
            
            # update total `new_cases` counter
            count_output_num = 0
            for des,des_num_case in new_cases_by_infectious_day.items():
                if des not in new_cases: new_cases[des] = 0
                prev_num = new_cases[des]
                new_cases[des] += des_num_case

                # update case level output
                for i in range(des_num_case):
                    case_level_output_str += '%d, %s, %d, %s, %d\n'%(simulation_day, cbg, src_case_id[count_output_num+i], des, prev_num + i)
                count_output_num += des_num_case


            # update for recovered cases
            if d == infectious_day-1:
                print (cbg_active_case[d])
                counter.loc[src_cbg_idx, 'recovered'] += cbg_active_case[d]
                counter.loc[src_cbg_idx, 'infectious'] -= cbg_active_case[d]
                cbg_active_case[0] = 0
                # update region level output on recovered cases
                region_level_output_str += '%d, %s, %d, %d, %d\n'%(simulation_day, cbg,
                                                                   counter.loc[src_cbg_idx,'susceptible'].item(),
                                                                   counter.loc[src_cbg_idx,'infectious'].item(),
                                                                   counter.loc[src_cbg_idx,'recovered'].item())
            
            if d != 0:
                cbg_active_case[d] = cbg_active_case[d-1]
                cbg_case_idx_lower[d] = cbg_case_idx_lower[d-1]
                
        
        # Clear the cbg from activate case tracker if no case active
        if sum(cbg_active_case) == 0: active_case.pop(cbg)
        
    # update region level output on recovered cases  
            
    # update active case
    for des,num_new_cases in new_cases.items():
        if des not in active_case: 
            active_case[des] = [[0] * infectious_day, [total_infected_tracker[des]] * infectious_day]
        active_case[des][0][0] = num_new_cases
        active_case[des][1][0] = total_infected_tracker[des]
        
        des_cbg_idx = counter[counter['GeoId'] == des].index
        counter.loc[des_cbg_idx, 'susceptible'] -= num_new_cases
        counter.loc[des_cbg_idx, 'infectious'] += num_new_cases
        total_infected_tracker[des] += num_new_cases
        
        # update region level output
        region_level_output_str += '%d, %s, %d, %d, %d\n'%(simulation_day, des,
                                                           counter.loc[des_cbg_idx,'susceptible'].item(),
                                                           counter.loc[des_cbg_idx,'infectious'].item(),
                                                           counter.loc[des_cbg_idx,'recovered'].item())
    return region_level_output_str, case_level_output_str

In [ ]:
def next_day(simulation_day):
    region_level_output_str = ''
    case_level_output_str = ''
    new_cases = {}
    for cbg in list(active_case.keys()):

        des_prob_df = spread_prob_grouped_df.get_group(cbg)
        cbg_active_case = active_case[cbg][0]
        cbg_case_idx_lower = active_case[cbg][1]
        src_cbg_idx = counter[counter['GeoId'] == cbg].index
        print(cbg_active_case)
        for d in range(infectious_day-1, -1, -1):

            num_infectious = cbg_active_case[d]

            # update for recovered cases
            if d == infectious_day-1:
                print (cbg_active_case[d])
                counter.loc[src_cbg_idx, 'recovered'] += cbg_active_case[d]
                counter.loc[src_cbg_idx, 'infectious'] -= cbg_active_case[d]
                # update region level output on recovered cases
                region_level_output_str += '%d, %s, %d, %d, %d\n'%(simulation_day, cbg,
                                                                   counter.loc[src_cbg_idx,'susceptible'].item(),
                                                                   counter.loc[src_cbg_idx,'infectious'].item(),
                                                                   counter.loc[src_cbg_idx,'recovered'].item())
            if d != 0:
                cbg_active_case[d] = cbg_active_case[d-1]
                cbg_case_idx_lower[d] = cbg_case_idx_lower[d-1]
            else:
                cbg_active_case[d] = 0

            if num_infectious == 0: continue

            # calculate number of new cases in infectious day 'd'
            num_new_cases = 0
            if num_infectious < 30:
                for case_idx in range(num_infectious):
                    if random.random() < infectious_rate[d]: num_new_cases += 1
            else:
                expected_cases = infectious_rate[d]*num_infectious
                if expected_cases >= 5:
                    num_new_cases = int(np.floor(max(np.random.normal(expected_cases,expected_cases*(1-infectious_rate[d])), 0)))
                else:
                    num_new_cases = np.random.poisson(expected_cases)
            if num_new_cases == 0: continue

            # calculate number of new cases in each possible infected cbg
            new_cases_by_infectious_day = {}
            num_new_cases_actual = 0
            if num_new_cases < 30: # Number of cases too small to use statistical methods
                for i in range(num_new_cases):
                    des_cbg = get_random_des(des_prob_df)
                    # Spread to a fully infected region, assume nothing happens
                    if counter[counter['GeoId'] == des_cbg]['susceptible'].item() == 0: continue
                    if des_cbg not in new_cases_by_infectious_day: new_cases_by_infectious_day[des_cbg] = 0
                    new_cases_by_infectious_day[des_cbg] += 1
                    num_new_cases_actual += 1
            else: # Approximate Binomial distribution with Normal distribution and Poisson distribution
                for i in des_prob_df.index:
                    des = des_prob_df.loc[i, 'to']
                    des_s_num = int(counter[counter['GeoId'] == des]['susceptible'].item())
                    if des_s_num == 0: continue

                    prob = des_prob_df.loc[i, 'prob']
                    expected_cases = num_new_cases * prob
                    expected_no_cases =  num_new_cases * (1-prob)
                    if expected_cases > 5 and expected_no_cases > 5:
                        var_case = expected_cases * (1-prob)
                        des_new_case = int(np.floor(np.random.normal(expected_cases,var_case)))
                    else:
                        des_new_case = np.random.poisson(expected_cases)

                    # Check if fully infected
                    des_new_case = min(des_s_num, des_new_case)
                    des_new_case = max(0,des_new_case)
                    if des_new_case > 0: new_cases_by_infectious_day[des] = des_new_case
                    num_new_cases_actual += des_new_case
            src_case_id = cbg_case_idx_lower[d] + np.random.choice(num_infectious, size = num_new_cases_actual, replace = False)

            # update total `new_cases` counter
            count_output_num = 0
            for des,des_num_case in new_cases_by_infectious_day.items():
                if des not in new_cases: new_cases[des] = 0
                prev_num = new_cases[des]
                new_cases[des] += des_num_case

                # update case level output
                for i in range(des_num_case):
                    case_level_output_str += '%d, %s, %d, %s, %d\n'%(simulation_day, cbg, src_case_id[count_output_num+i], des, prev_num + i)
                count_output_num += des_num_case


            # update for recovered cases
            if d == infectious_day-1:
                print (cbg_active_case[d])
                counter.loc[src_cbg_idx, 'recovered'] += cbg_active_case[d]
                counter.loc[src_cbg_idx, 'infectious'] -= cbg_active_case[d]
                cbg_active_case[0] = 0
                # update region level output on recovered cases
                region_level_output_str += '%d, %s, %d, %d, %d\n'%(simulation_day, cbg,
                                                                   counter.loc[src_cbg_idx,'susceptible'].item(),
                                                                   counter.loc[src_cbg_idx,'infectious'].item(),
                                                                   counter.loc[src_cbg_idx,'recovered'].item())

            if d != 0:
                cbg_active_case[d] = cbg_active_case[d-1]
                cbg_case_idx_lower[d] = cbg_case_idx_lower[d-1]


        # Clear the cbg from activate case tracker if no case active
        if sum(cbg_active_case) == 0: active_case.pop(cbg)

    # update region level output on recovered cases

    # update active case
    for des,num_new_cases in new_cases.items():
        if des not in active_case:
            active_case[des] = [[0] * infectious_day, [total_infected_tracker[des]] * infectious_day]
        active_case[des][0][0] = num_new_cases
        active_case[des][1][0] = total_infected_tracker[des]

        des_cbg_idx = counter[counter['GeoId'] == des].index
        counter.loc[des_cbg_idx, 'susceptible'] -= num_new_cases
        counter.loc[des_cbg_idx, 'infectious'] += num_new_cases
        total_infected_tracker[des] += num_new_cases

        # update region level output
        region_level_output_str += '%d, %s, %d, %d, %d\n'%(simulation_day, des,
                                                           counter.loc[des_cbg_idx,'susceptible'].item(),
                                                           counter.loc[des_cbg_idx,'infectious'].item(),
                                                           counter.loc[des_cbg_idx,'recovered'].item())
    return region_level_output_str, case_level_output_str

In [175]:
for i in range(7):
    next_day(i)

[500, 0, 0, 0, 0, 0, 0]
0
[500, 0, 0, 0, 0, 0, 0]
0
[500, 0, 0, 0, 0, 0, 0]
0
[500, 0, 0, 0, 0, 0, 0]
0
[500, 0, 0, 0, 0, 0, 0]
0
[0, 500, 0, 0, 0, 0, 0]
0
[1, 500, 0, 0, 0, 0, 0]
0
[5, 500, 0, 0, 0, 0, 0]
0
[0, 500, 0, 0, 0, 0, 0]
0
[3, 500, 0, 0, 0, 0, 0]
0
[1, 0, 0, 0, 0, 0, 0]
0
[1, 0, 0, 0, 0, 0, 0]
0
[55, 0, 0, 0, 0, 0, 0]
0
[1, 0, 0, 0, 0, 0, 0]
0
[1, 0, 0, 0, 0, 0, 0]
0
[1, 0, 0, 0, 0, 0, 0]
0
[2, 0, 0, 0, 0, 0, 0]
0
[1, 0, 0, 0, 0, 0, 0]
0
[3, 0, 0, 0, 0, 0, 0]
0
[1, 0, 0, 0, 0, 0, 0]
0
[1, 0, 0, 0, 0, 0, 0]
0
[2, 0, 0, 0, 0, 0, 0]
0
[1, 0, 0, 0, 0, 0, 0]
0
[1, 0, 0, 0, 0, 0, 0]
0
[2, 0, 0, 0, 0, 0, 0]
0
[2, 0, 0, 0, 0, 0, 0]
0
[1, 0, 0, 0, 0, 0, 0]
0
[1, 0, 0, 0, 0, 0, 0]
0
[1, 0, 0, 0, 0, 0, 0]
0
[1, 0, 0, 0, 0, 0, 0]
0
[3, 0, 0, 0, 0, 0, 0]
0
[1, 0, 0, 0, 0, 0, 0]
0
[2, 0, 0, 0, 0, 0, 0]
0
[2, 0, 0, 0, 0, 0, 0]
0
[2, 0, 0, 0, 0, 0, 0]
0
[2, 0, 0, 0, 0, 0, 0]
0
[2, 0, 0, 0, 0, 0, 0]
0
[1, 0, 0, 0, 0, 0, 0]
0
[1, 0, 0, 0, 0, 0, 0]
0
[1, 0, 0, 0, 0, 0, 0]
0
[1, 0, 0, 0, 0, 0, 

In [176]:
counter

,day,GeoId,susceptible,infectious,recovered
0,0,010010201001,730,0,0
1,0,010010201002,1263,0,0
2,0,010010202001,835,0,0
3,0,010010202002,1124,0,0
4,0,010010203001,2774,0,0
...,...,...,...,...,...
220329,0,721537506011,883,0,0
220330,0,721537506012,2523,0,0
220331,0,721537506013,991,0,0
220332,0,721537506021,1577,0,0


In [56]:
for k,v in active_case.items():
    print(k, v[0], v[1])

220450309003 [1, 0, 0, 0, 0, 0, 0] [71353    517
Name: Infected, dtype: int64, 0, 0, 0, 0, 0, 0]
261251605002 [500, 0, 0, 0, 0, 0, 0] [0, 0, 0, 0, 0, 0, 0]
420270115022 [500, 0, 0, 0, 0, 0, 0] [0, 0, 0, 0, 0, 0, 0]
121270832082 [500, 0, 0, 0, 0, 0, 0] [0, 0, 0, 0, 0, 0, 0]
180890413021 [14, 0, 0, 0, 0, 0, 0] [71353    570
Name: Infected, dtype: int64, 0, 0, 0, 0, 0, 0]
220450311002 [1, 0, 0, 0, 0, 0, 0] [71353    689
Name: Infected, dtype: int64, 500, 500, 500, 500, 500, 500]
220450302004 [3, 0, 0, 0, 0, 0, 0] [71353    503
Name: Infected, dtype: int64, 503, 503, 503, 503, 503, 503]
220450309002 [1, 0, 0, 0, 0, 0, 0] [71353    506
Name: Infected, dtype: int64, 506, 506, 506, 506, 506, 506]
220450303022 [1, 0, 0, 0, 0, 0, 0] [71353    507
Name: Infected, dtype: int64, 507, 507, 507, 507, 507, 507]
220450304005 [1, 0, 0, 0, 0, 0, 0] [71353    690
Name: Infected, dtype: int64, 508, 508, 508, 508, 508, 508]
220450305003 [1, 0, 0, 0, 0, 0, 0] [71353    510
Name: Infected, dtype: int64, 510,

In [50]:
print(case_level_output_str)

0, 000000000000, -1, 220450309003, 0
0, 000000000000, -1, 220450309003, 1
0, 000000000000, -1, 220450309003, 2
0, 000000000000, -1, 220450309003, 3
0, 000000000000, -1, 220450309003, 4
0, 000000000000, -1, 220450309003, 5
0, 000000000000, -1, 220450309003, 6
0, 000000000000, -1, 220450309003, 7
0, 000000000000, -1, 220450309003, 8
0, 000000000000, -1, 220450309003, 9
0, 000000000000, -1, 220450309003, 10
0, 000000000000, -1, 220450309003, 11
0, 000000000000, -1, 220450309003, 12
0, 000000000000, -1, 220450309003, 13
0, 000000000000, -1, 220450309003, 14
0, 000000000000, -1, 220450309003, 15
0, 000000000000, -1, 220450309003, 16
0, 000000000000, -1, 220450309003, 17
0, 000000000000, -1, 220450309003, 18
0, 000000000000, -1, 220450309003, 19
0, 000000000000, -1, 220450309003, 20
0, 000000000000, -1, 220450309003, 21
0, 000000000000, -1, 220450309003, 22
0, 000000000000, -1, 220450309003, 23
0, 000000000000, -1, 220450309003, 24
0, 000000000000, -1, 220450309003, 25
0, 000000000000, -1, 2

In [130]:
def next_day(simulation_day):
    region_level_output_str = ''
    case_level_output_str = ''
    new_cases = {}
    for cbg in list(active_case.keys()):
        
        # calculate number of new cases
        num_new_cases = 0
        
        for i in range(infectious_day-1, -1, -1):
            mean_num_case = infectious_rate[i]*active_case[cbg][i]
            if mean_num_case != 0: num_new_cases +=  max(np.random.normal(mean_num_case,mean_num_case*(1-infectious_rate[i])), 0)
            # update active case
            active_case[cbg][i] = active_case[cbg][i-1] if i > 0 else 0
        num_new_cases = int(np.floor(num_new_cases))
        
        # calculate number of new cases in each possible infected cbg
        des_prob_df = spread_prob_grouped_df.get_group(cbg)
        if num_new_cases < 30:
            
            
            
            for i in range(num_new_cases):
                des_cbg = get_random_des(des_prob_df)
                
                # Spread to a fully infected region, assume nothing happens
                if counter[counter['GeoId'] == des_cbg]['susceptible'] == 0: continue
                
                if des_cbg not in new_cases: new_cases[des_cbg] = 0
                new_cases[des_cbg] += 1
        else:
            for i in des_prob_df.index:
                des = des_prob_df.loc[i, 'to']
                prob = des_prob_df.loc[i, 'prob']
                expected_case = num_new_cases * prob
                expected_no_case =  num_new_cases * (1-prob)
                des_new_case = 0
                if expected_case > 5 and expected_no_case > 5:
                    var_case = expected_case * (1-prob)
                    des_new_case = np.floor(np.random.normal(expected_case,var_case))
                else:
                    des_new_case = np.random.poisson(expected_case)
                if des_new_case > 0:
                    if des not in new_cases: new_cases[des] = 0
                    new_cases[des] += des_new_case
        

    
    
    
    # update active case
    for des,num_new_case in new_cases.items():
        if des not in active_case: active_case[des] = np.zeros(infectious_day)
        active_case[des][0] = num_new_case
        
        # update region level output
        des_idx = counter[counter['GeoId'] == des]
        region_level_output_str += '%d, %s, %d, %d, %d\n'%(simulation_day, des_idx, counter.loc[des_idx,'susceptible'].item(), counter.loc[des_idx,'infectious'].item(), counter.loc[des_idx,'recovered'].item())

In [72]:
next_day()

In [73]:
active_case

{'220450309003': array([  1., 100.,   0.,   0.,   0.,   0.,   0.]),
 '261251605002': array([  0., 100.,   0.,   0.,   0.,   0.,   0.]),
 '420270115022': array([  0., 100.,   0.,   0.,   0.,   0.,   0.]),
 '121270832082': array([  1., 100.,   0.,   0.,   0.,   0.,   0.]),
 '180890413021': array([  0., 100.,   0.,   0.,   0.,   0.,   0.]),
 '220450302003': array([1., 0., 0., 0., 0., 0., 0.]),
 '220450303023': array([2., 0., 0., 0., 0., 0., 0.]),
 '220450303011': array([1., 0., 0., 0., 0., 0., 0.]),
 '220450301001': array([3., 0., 0., 0., 0., 0., 0.]),
 '220450312001': array([2., 0., 0., 0., 0., 0., 0.]),
 '220450302004': array([1., 0., 0., 0., 0., 0., 0.]),
 '220450305003': array([1., 0., 0., 0., 0., 0., 0.]),
 '220450306004': array([1., 0., 0., 0., 0., 0., 0.]),
 '220450304005': array([1., 0., 0., 0., 0., 0., 0.]),
 '220450311003': array([2., 0., 0., 0., 0., 0., 0.]),
 '220450312002': array([1., 0., 0., 0., 0., 0., 0.]),
 '220450310004': array([1., 0., 0., 0., 0., 0., 0.]),
 '2204503110

In [75]:
next_day()
active_case

{'220450309003': array([  1.,   0.,   1., 100.,   0.,   0.,   0.]),
 '261251605002': array([  0.,   0.,   0., 100.,   0.,   0.,   0.]),
 '420270115022': array([  1.,   0.,   0., 100.,   0.,   0.,   0.]),
 '121270832082': array([  1.,   0.,   1., 100.,   0.,   0.,   0.]),
 '180890413021': array([  0.,   0.,   0., 100.,   0.,   0.,   0.]),
 '220450302003': array([0., 1., 1., 0., 0., 0., 0.]),
 '220450303023': array([0., 1., 2., 0., 0., 0., 0.]),
 '220450303011': array([3., 1., 1., 0., 0., 0., 0.]),
 '220450301001': array([3., 1., 3., 0., 0., 0., 0.]),
 '220450312001': array([5., 1., 2., 0., 0., 0., 0.]),
 '220450302004': array([2., 0., 1., 0., 0., 0., 0.]),
 '220450305003': array([1., 2., 1., 0., 0., 0., 0.]),
 '220450306004': array([3., 2., 1., 0., 0., 0., 0.]),
 '220450304005': array([2., 2., 1., 0., 0., 0., 0.]),
 '220450311003': array([0., 1., 2., 0., 0., 0., 0.]),
 '220450312002': array([2., 0., 1., 0., 0., 0., 0.]),
 '220450310004': array([3., 1., 1., 0., 0., 0., 0.]),
 '2204503110